# Sessão 5 – Orquestrador Multi-Agente

Demonstra uma pipeline simples de dois agentes (Investigador -> Editor) utilizando Foundry Local.


### Explicação: Instalação de Dependências
Instala `foundry-local-sdk` e `openai`, necessários para acesso a modelos locais e completões de chat. Idempotente.


# Cenário
Implementa um padrão minimalista de orquestração com dois agentes:
- **Agente Pesquisador** recolhe pontos factuais concisos
- **Agente Editor** reescreve com clareza executiva

Demonstra memória partilhada por agente, passagem sequencial de resultados intermédios e uma função de pipeline simples. Pode ser expandido para mais funções (ex.: Crítico, Verificador) ou ramos paralelos.

**Variáveis de Ambiente:**
- `FOUNDRY_LOCAL_ALIAS` - Modelo padrão a utilizar (predefinição: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Modelo principal do agente (substitui ALIAS)
- `AGENT_MODEL_EDITOR` - Modelo do agente editor (predefinição: modelo principal)

**Referência SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Como funciona:**
1. **FoundryLocalManager** inicia automaticamente o serviço Foundry Local
2. Faz o download e carrega o modelo especificado (ou utiliza a versão em cache)
3. Fornece um endpoint compatível com OpenAI para interação
4. Cada agente pode utilizar um modelo diferente para tarefas especializadas
5. Lógica de repetição integrada lida com falhas transitórias de forma eficiente

**Principais Funcionalidades:**
- ✅ Descoberta e inicialização automática do serviço
- ✅ Gestão do ciclo de vida do modelo (download, cache, carregamento)
- ✅ Compatibilidade com o SDK OpenAI para uma API familiar
- ✅ Suporte a múltiplos modelos para especialização de agentes
- ✅ Tratamento robusto de erros com lógica de repetição
- ✅ Inferência local (não requer API na nuvem)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Explicação: Importações Principais e Tipagem
Introduz dataclasses para armazenamento de mensagens de agentes e dicas de tipagem para maior clareza. Importa o gestor local do Foundry e o cliente OpenAI para ações subsequentes do agente.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Explicação: Inicialização do Modelo (Padrão SDK)
Utiliza o Foundry Local Python SDK para uma gestão robusta de modelos:
- **FoundryLocalManager(alias)** - Inicia automaticamente o serviço e carrega o modelo pelo alias
- **get_model_info(alias)** - Resolve o alias para um ID de modelo concreto
- **manager.endpoint** - Fornece o endpoint do serviço para o cliente OpenAI
- **manager.api_key** - Fornece a chave API (opcional para uso local)
- Suporta modelos separados para diferentes agentes (primário vs editor)
- Lógica de repetição integrada com recuo exponencial para maior resiliência
- Verificação de conexão para garantir que o serviço está pronto

**Padrão SDK Principal:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Gestão de Ciclo de Vida:**
- Os managers são armazenados globalmente para uma limpeza adequada
- Cada agente pode usar um modelo diferente para especialização
- Descoberta automática de serviços e gestão de conexões
- Repetição graciosa com recuo exponencial em caso de falhas

Isto garante uma inicialização adequada antes de começar a orquestração dos agentes.

**Referência:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Explicação: Classes Agent & Memory
Define `AgentMsg` leve para entradas de memória e `Agent`, que encapsula:
- **Função do sistema** - Persona e instruções do agente
- **Histórico de mensagens** - Mantém o contexto da conversa
- **Método act()** - Executa ações com tratamento adequado de erros

O agente pode usar diferentes modelos (principal vs editor) e mantém um contexto isolado por agente. Este padrão permite:
- Persistência de memória entre ações
- Atribuição flexível de modelos por agente
- Isolamento e recuperação de erros
- Encadeamento e orquestração simplificados


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Explicação: Pipeline Orquestrado
Cria dois agentes especializados:
- **Investigador**: Utiliza o modelo principal, recolhe informações factuais
- **Editor**: Pode usar um modelo separado (se configurado), refina e reescreve

A função `pipeline`:
1. O Investigador recolhe informações brutas
2. O Editor refina e transforma num resultado pronto para apresentação
3. Retorna tanto os resultados intermédios como os finais

Este padrão permite:
- Especialização de modelos (diferentes modelos para diferentes funções)
- Melhoria da qualidade através de processamento em várias etapas
- Rastreabilidade da transformação da informação
- Extensão fácil para mais agentes ou processamento paralelo


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Explicação: Execução do Pipeline & Resultados
Executa o pipeline multi-agente numa questão com foco em conformidade + latência para demonstrar:
- Transformação de informação em várias etapas
- Especialização e colaboração entre agentes
- Melhoria da qualidade do output através de refinamento
- Rastreabilidade (preservação de outputs intermediários e finais)

**Estrutura dos Resultados:**
- `question` - Consulta original do utilizador
- `research` - Resultado bruto da pesquisa (pontos factuais)
- `final` - Resumo executivo refinado
- `models` - Modelos utilizados em cada etapa

**Ideias de Extensão:**
1. Adicionar um agente Crítico para revisão de qualidade
2. Implementar agentes de pesquisa paralelos para diferentes aspetos
3. Adicionar um agente Verificador para verificação de factos
4. Utilizar modelos diferentes para níveis de complexidade distintos
5. Implementar ciclos de feedback para melhoria iterativa


### Avançado: Configuração Personalizada de Agente

Experimente personalizar o comportamento do agente modificando as variáveis de ambiente antes de executar a célula de inicialização:

**Modelos Disponíveis:**
- Utilize `foundry model ls` no terminal para ver todos os modelos disponíveis
- Exemplos: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, etc.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Aviso**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se uma tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
